In [ ]:
import pandas as pd
import numpy as np
import random
import os
from sklearn.preprocessing import LabelEncoder
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [ ]:
import datetime
import random
import os
import sys
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib
from matplotlib import rc
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(0)

In [ ]:
train_df = pd.read_csv('data/train.csv')
test_df = pd.read_csv('data/test.csv')

In [ ]:
import numpy as np
from supervised import AutoML

In [ ]:
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])

In [ ]:
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])

# 군과 구를 나눠줍니다

In [ ]:
train_df['군'] = ''
train_df['구'] = ''

test_df['군'] = ''
test_df['구'] = ''

# 주소 분리 및 군과 구 열에 저장
for i in range(0, len(train_df)):
    address = train_df['시군구'][i]
    elements = address.split(" ")
    if len(elements) == 3:
        city = elements[0]
        county = elements[1]
        district = elements[2]
        train_df.at[i, '군'] = county
        train_df.at[i, '구'] = district
    else:
        print(f"잘못된 주소 형식입니다: {address}")

# 결과 출력
print(train_df)

                   ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_00000 2019-01-01 00:00:00  화요일   맑음     대구광역시 중구 대신동   
1      ACCIDENT_00001 2019-01-01 00:00:00  화요일   흐림    대구광역시 달서구 감삼동   
2      ACCIDENT_00002 2019-01-01 01:00:00  화요일   맑음    대구광역시 수성구 두산동   
3      ACCIDENT_00003 2019-01-01 02:00:00  화요일   맑음     대구광역시 북구 복현동   
4      ACCIDENT_00004 2019-01-01 04:00:00  화요일   맑음     대구광역시 동구 신암동   
...               ...                 ...  ...  ...              ...   
39604  ACCIDENT_39604 2021-12-31 19:00:00  금요일   맑음  대구광역시 수성구 수성동3가   
39605  ACCIDENT_39605 2021-12-31 19:00:00  금요일   맑음    대구광역시 달서구 상인동   
39606  ACCIDENT_39606 2021-12-31 21:00:00  금요일   맑음    대구광역시 달서구 월성동   
39607  ACCIDENT_39607 2021-12-31 22:00:00  금요일   맑음     대구광역시 달서구 장동   
39608  ACCIDENT_39608 2021-12-31 23:00:00  금요일   맑음     대구광역시 서구 비산동   

                  도로형태 노면상태  사고유형 사고유형 - 세부분류       법규위반  ... 피해운전자 성별  \
0             단일로 - 기타   건조  차대사람  길가장자리구역통행중    안전운전불이행  ...

In [ ]:
for i in range(0, len(test_df)):
    address = test_df['시군구'][i]
    elements = address.split(" ")
    if len(elements) == 3:
        city = elements[0]
        county = elements[1]
        district = elements[2]
        test_df.at[i, '군'] = county
        test_df.at[i, '구'] = district
    else:
        print(f"잘못된 주소 형식입니다: {address}")

# 결과 출력
print(test_df)

                   ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_39609 2022-01-01 01:00:00  토요일   맑음     대구광역시 수성구 상동   
1      ACCIDENT_39610 2022-01-01 01:00:00  토요일   맑음    대구광역시 수성구 지산동   
2      ACCIDENT_39611 2022-01-01 04:00:00  토요일   맑음  대구광역시 수성구 수성동2가   
3      ACCIDENT_39612 2022-01-01 04:00:00  토요일   맑음    대구광역시 수성구 신매동   
4      ACCIDENT_39613 2022-01-01 06:00:00  토요일   맑음    대구광역시 달서구 감삼동   
...               ...                 ...  ...  ...              ...   
10958  ACCIDENT_50567 2022-12-31 18:00:00  토요일   맑음     대구광역시 남구 대명동   
10959  ACCIDENT_50568 2022-12-31 18:00:00  토요일   맑음    대구광역시 수성구 시지동   
10960  ACCIDENT_50569 2022-12-31 20:00:00  토요일   맑음    대구광역시 수성구 연호동   
10961  ACCIDENT_50570 2022-12-31 20:00:00  토요일   맑음    대구광역시 수성구 범물동   
10962  ACCIDENT_50571 2022-12-31 21:00:00  토요일   맑음     대구광역시 동구 효목동   

              도로형태 노면상태  사고유형    군      구  
0       교차로 - 교차로안   건조  차대사람  수성구     상동  
1         단일로 - 기타   건조  차대사람  수성구    지산동  
2  

# 외부 데이터 사용

이 부분은 이희원님의 코드를 참고하였습니다.

In [ ]:
light_df = pd.read_csv('data/external_open/대구 보안등 정보.csv', encoding='cp949')[['설치개수', '소재지지번주소']]

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

light_df[['도시', '군', '구', '번지']] = light_df['소재지지번주소'].str.extract(location_pattern)
light_df = light_df.drop(columns=['소재지지번주소', '번지'])

light_df = light_df.groupby(['도시', '군', '구']).sum().reset_index()
light_df.reset_index(inplace=True, drop=True)

In [ ]:
parking_df = pd.read_csv('data/external_open/대구 주차장 정보.csv', encoding='cp949')[['소재지지번주소', '급지구분']]
parking_df = pd.get_dummies(parking_df, columns=['급지구분'])

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

parking_df[['도시', '군', '구', '번지']] = parking_df['소재지지번주소'].str.extract(location_pattern)
parking_df = parking_df.drop(columns=['소재지지번주소', '번지'])

parking_df = parking_df.groupby(['도시', '군', '구']).sum().reset_index()
parking_df.reset_index(inplace=True, drop=True)

In [ ]:
child_area_df = pd.read_csv('data/external_open/대구 어린이 보호 구역 정보.csv', encoding='cp949').drop_duplicates()[['소재지지번주소']]
child_area_df['cnt'] = 1

location_pattern = r'(\S+) (\S+) (\S+) (\S+)'

child_area_df[['도시', '군', '구', '번지']] = child_area_df['소재지지번주소'].str.extract(location_pattern)
child_area_df = child_area_df.drop(columns=['소재지지번주소', '번지'])

child_area_df = child_area_df.groupby(['도시', '군', '구']).sum().reset_index()
child_area_df.reset_index(inplace=True, drop=True)

In [ ]:
train_df = pd.merge(train_df, light_df, how='left', on=['군', '구'])
train_df = pd.merge(train_df, child_area_df, how='left', on=['군', '구'])
train_df = pd.merge(train_df, parking_df, how='left', on=['군', '구'])

test_df = pd.merge(test_df, light_df, how='left', on=['군', '구'])
test_df = pd.merge(test_df, child_area_df, how='left', on=['군', '구'])
test_df = pd.merge(test_df, parking_df, how='left', on=['군', '구'])

In [ ]:
test_df['군'].unique()

array(['수성구', '달서구', '중구', '동구', '서구', '북구', '남구', '달성군'], dtype=object)

## One-hot-encoding

In [ ]:
from sklearn.preprocessing import OneHotEncoder

# train_df와 test_df의 기상상태 및 노면상태 열 선택
train_categorical_data = train_df[['군']]
test_categorical_data = test_df[['군']]

# OneHotEncoder 인스턴스 생성 및 fit_transform 수행
encoder = OneHotEncoder()
train_encoded = encoder.fit_transform(train_categorical_data)
test_encoded = encoder.transform(test_categorical_data)

# OneHotEncoder가 사용한 카테고리 목록을 가져와서 카테고리 이름을 열 이름으로 변환
feature_names = encoder.get_feature_names_out(['군'])

# 밀집 행렬로 변환 (선택 사항)
train_encoded_dense = train_encoded.toarray()
test_encoded_dense = test_encoded.toarray()

# 데이터프레임으로 변환 (선택 사항)
import pandas as pd

train_encoded_df = pd.DataFrame(train_encoded_dense, columns=feature_names, index=train_df.index)
test_encoded_df = pd.DataFrame(test_encoded_dense, columns=feature_names, index=test_df.index)

# 기존 열 제거
train_df = train_df.drop(['군', ], axis=1)
test_df = test_df.drop(['군'], axis=1)

# 인코딩된 열 추가
train_df = pd.concat([train_df, train_encoded_df], axis=1)
test_df = pd.concat([test_df, test_encoded_df], axis=1)


In [ ]:
train_df

ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_00000 2019-01-01 00:00:00  화요일   맑음     대구광역시 중구 대신동   
1      ACCIDENT_00001 2019-01-01 00:00:00  화요일   흐림    대구광역시 달서구 감삼동   
2      ACCIDENT_00002 2019-01-01 01:00:00  화요일   맑음    대구광역시 수성구 두산동   
3      ACCIDENT_00003 2019-01-01 02:00:00  화요일   맑음     대구광역시 북구 복현동   
4      ACCIDENT_00004 2019-01-01 04:00:00  화요일   맑음     대구광역시 동구 신암동   
...               ...                 ...  ...  ...              ...   
39604  ACCIDENT_39604 2021-12-31 19:00:00  금요일   맑음  대구광역시 수성구 수성동3가   
39605  ACCIDENT_39605 2021-12-31 19:00:00  금요일   맑음    대구광역시 달서구 상인동   
39606  ACCIDENT_39606 2021-12-31 21:00:00  금요일   맑음    대구광역시 달서구 월성동   
39607  ACCIDENT_39607 2021-12-31 22:00:00  금요일   맑음     대구광역시 달서구 장동   
39608  ACCIDENT_39608 2021-12-31 23:00:00  금요일   맑음     대구광역시 서구 비산동   

                  도로형태 노면상태  사고유형 사고유형 - 세부분류       법규위반  ... 급지구분_2 급지구분_3  \
0             단일로 - 기타   건조  차대사람  길가장자리구역통행중    안전운전불이행  ...    0.0    0.0   
1             단일로 - 기타   건조  차대사람       보도통행중         기타  ...    1.0    3.0   
2             단일로 - 기타   건조  차대사람       차도통행중    안전운전불이행  ...    NaN    NaN   
3             단일로 - 기타   건조   차대차          추돌    안전운전불이행  ...    9.0    5.0   
4             단일로 - 기타   건조   차대차          추돌    안전운전불이행  ...    1.0    0.0   
...                ...  ...   ...         ...        ...  ...    ...    ...   
39604       교차로 - 교차로안   건조   차대차        측면충돌       신호위반  ...    NaN    NaN   
39605         단일로 - 기타   건조   차대차        측면충돌    안전거리미확보  ...    0.0    5.0   
39606       교차로 - 교차로안   건조   차대차        측면충돌  교차로운행방법위반  ...    1.0    0.0   
39607          기타 - 기타   건조   차대차          추돌    안전운전불이행  ...    0.0    1.0   
39608  단일로 - 지하차도(도로)내   건조   차대차        측면충돌    안전운전불이행  ...   30.0   13.0   

      군_남구 군_달서구 군_달성군 군_동구 군_북구 군_서구  군_수성구  군_중구  
0      0.0   0.0   0.0  0.0  0.0  0.0    0.0   1.0  
1      0.0   1.0   0.0  0.0  0.0  0.0    0.0   0.0  
2      0.0   0.0   0.0  0.0  0.0  0.0    1.0   0.0  
3      0.0   0.0   0.0  0.0  1.0  0.0    0.0   0.0  
4      0.0   0.0   0.0  1.0  0.0  0.0    0.0   0.0  
...    ...   ...   ...  ...  ...  ...    ...   ...  
39604  0.0   0.0   0.0  0.0  0.0  0.0    1.0   0.0  
39605  0.0   1.0   0.0  0.0  0.0  0.0    0.0   0.0  
39606  0.0   1.0   0.0  0.0  0.0  0.0    0.0   0.0  
39607  0.0   1.0   0.0  0.0  0.0  0.0    0.0   0.0  
39608  0.0   0.0   0.0  0.0  0.0  1.0    0.0   0.0  

[39609 rows x 40 columns]

# 시군구별 발생횟수 feature 추가

In [ ]:
accident_counts = train_df['시군구'].value_counts().reset_index()
accident_counts.columns = ['시군구', '사고횟수']

In [ ]:
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
train_df['사고발생횟수'] = train_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])


In [ ]:
# '시군구'별 사고 횟수를 train_df에 매핑하기 위해 '시군구' 컬럼을 기준으로 사고 횟수를 가져옵니다.
test_df['사고발생횟수'] = test_df['시군구'].map(accident_counts.set_index('시군구')['사고횟수'])


In [ ]:
train_df

ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_00000 2019-01-01 00:00:00  화요일   맑음     대구광역시 중구 대신동   
1      ACCIDENT_00001 2019-01-01 00:00:00  화요일   흐림    대구광역시 달서구 감삼동   
2      ACCIDENT_00002 2019-01-01 01:00:00  화요일   맑음    대구광역시 수성구 두산동   
3      ACCIDENT_00003 2019-01-01 02:00:00  화요일   맑음     대구광역시 북구 복현동   
4      ACCIDENT_00004 2019-01-01 04:00:00  화요일   맑음     대구광역시 동구 신암동   
...               ...                 ...  ...  ...              ...   
39604  ACCIDENT_39604 2021-12-31 19:00:00  금요일   맑음  대구광역시 수성구 수성동3가   
39605  ACCIDENT_39605 2021-12-31 19:00:00  금요일   맑음    대구광역시 달서구 상인동   
39606  ACCIDENT_39606 2021-12-31 21:00:00  금요일   맑음    대구광역시 달서구 월성동   
39607  ACCIDENT_39607 2021-12-31 22:00:00  금요일   맑음     대구광역시 달서구 장동   
39608  ACCIDENT_39608 2021-12-31 23:00:00  금요일   맑음     대구광역시 서구 비산동   

                  도로형태 노면상태  사고유형 사고유형 - 세부분류       법규위반  ... 급지구분_3 군_남구  \
0             단일로 - 기타   건조  차대사람  길가장자리구역통행중    안전운전불이행  ...    0.0  0.0   
1             단일로 - 기타   건조  차대사람       보도통행중         기타  ...    3.0  0.0   
2             단일로 - 기타   건조  차대사람       차도통행중    안전운전불이행  ...    NaN  0.0   
3             단일로 - 기타   건조   차대차          추돌    안전운전불이행  ...    5.0  0.0   
4             단일로 - 기타   건조   차대차          추돌    안전운전불이행  ...    0.0  0.0   
...                ...  ...   ...         ...        ...  ...    ...  ...   
39604       교차로 - 교차로안   건조   차대차        측면충돌       신호위반  ...    NaN  0.0   
39605         단일로 - 기타   건조   차대차        측면충돌    안전거리미확보  ...    5.0  0.0   
39606       교차로 - 교차로안   건조   차대차        측면충돌  교차로운행방법위반  ...    0.0  0.0   
39607          기타 - 기타   건조   차대차          추돌    안전운전불이행  ...    1.0  0.0   
39608  단일로 - 지하차도(도로)내   건조   차대차        측면충돌    안전운전불이행  ...   13.0  0.0   

      군_달서구 군_달성군 군_동구 군_북구 군_서구 군_수성구  군_중구  사고발생횟수  
0       0.0   0.0  0.0  0.0  0.0   0.0   1.0     131  
1       1.0   0.0  0.0  0.0  0.0   0.0   0.0     678  
2       0.0   0.0  0.0  0.0  0.0   1.0   0.0     604  
3       0.0   0.0  0.0  1.0  0.0   0.0   0.0     426  
4       0.0   0.0  1.0  0.0  0.0   0.0   0.0     825  
...     ...   ...  ...  ...  ...   ...   ...     ...  
39604   0.0   0.0  0.0  0.0  0.0   1.0   0.0     104  
39605   1.0   0.0  0.0  0.0  0.0   0.0   0.0    1143  
39606   1.0   0.0  0.0  0.0  0.0   0.0   0.0     618  
39607   1.0   0.0  0.0  0.0  0.0   0.0   0.0     125  
39608   0.0   0.0  0.0  0.0  1.0   0.0   0.0     968  

[39609 rows x 41 columns]

In [ ]:
test_df

ID                사고일시   요일 기상상태              시군구  \
0      ACCIDENT_39609 2022-01-01 01:00:00  토요일   맑음     대구광역시 수성구 상동   
1      ACCIDENT_39610 2022-01-01 01:00:00  토요일   맑음    대구광역시 수성구 지산동   
2      ACCIDENT_39611 2022-01-01 04:00:00  토요일   맑음  대구광역시 수성구 수성동2가   
3      ACCIDENT_39612 2022-01-01 04:00:00  토요일   맑음    대구광역시 수성구 신매동   
4      ACCIDENT_39613 2022-01-01 06:00:00  토요일   맑음    대구광역시 달서구 감삼동   
...               ...                 ...  ...  ...              ...   
10958  ACCIDENT_50567 2022-12-31 18:00:00  토요일   맑음     대구광역시 남구 대명동   
10959  ACCIDENT_50568 2022-12-31 18:00:00  토요일   맑음    대구광역시 수성구 시지동   
10960  ACCIDENT_50569 2022-12-31 20:00:00  토요일   맑음    대구광역시 수성구 연호동   
10961  ACCIDENT_50570 2022-12-31 20:00:00  토요일   맑음    대구광역시 수성구 범물동   
10962  ACCIDENT_50571 2022-12-31 21:00:00  토요일   맑음     대구광역시 동구 효목동   

              도로형태 노면상태  사고유형      구   도시_x  ...  급지구분_3 군_남구  군_달서구 군_달성군  \
0       교차로 - 교차로안   건조  차대사람     상동  대구광역시  ...     NaN  0.0    0.0   0.0   
1         단일로 - 기타   건조  차대사람    지산동    NaN  ...     2.0  0.0    0.0   0.0   
2       교차로 - 교차로안   건조   차대차  수성동2가    NaN  ...     NaN  0.0    0.0   0.0   
3         단일로 - 기타   건조   차대차    신매동    NaN  ...     1.0  0.0    0.0   0.0   
4       교차로 - 교차로안   건조   차대차    감삼동  대구광역시  ...     3.0  0.0    1.0   0.0   
...            ...  ...   ...    ...    ...  ...     ...  ...    ...   ...   
10958     단일로 - 터널   건조   차대차    대명동  대구광역시  ...     0.0  1.0    0.0   0.0   
10959     단일로 - 기타   건조   차대차    시지동    NaN  ...     NaN  0.0    0.0   0.0   
10960     단일로 - 기타   건조   차대차    연호동    NaN  ...     NaN  0.0    0.0   0.0   
10961  교차로 - 교차로부근   건조   차대차    범물동    NaN  ...     NaN  0.0    0.0   0.0   
10962  교차로 - 교차로부근   건조   차대차    효목동  대구광역시  ...     NaN  0.0    0.0   0.0   

       군_동구  군_북구  군_서구  군_수성구  군_중구  사고발생횟수  
0       0.0   0.0   0.0    1.0   0.0     338  
1       0.0   0.0   0.0    1.0   0.0     492  
2       0.0   0.0   0.0    1.0   0.0     147  
3       0.0   0.0   0.0    1.0   0.0     372  
4       0.0   0.0   0.0    0.0   0.0     678  
...     ...   ...   ...    ...   ...     ...  
10958   0.0   0.0   0.0    0.0   0.0    1749  
10959   0.0   0.0   0.0    1.0   0.0     167  
10960   0.0   0.0   0.0    1.0   0.0     107  
10961   0.0   0.0   0.0    1.0   0.0     239  
10962   1.0   0.0   0.0    0.0   0.0     597  

[10963 rows x 26 columns]

In [ ]:
# datetime 컬럼 처리
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])

# datetime을 여러 파생 변수로 변환
for df in [train_df, test_df]:
    df['year'] = df['사고일시'].dt.year
    df['month'] = df['사고일시'].dt.month
    df['day'] = df['사고일시'].dt.day
    df['hour'] = df['사고일시'].dt.hour
    df['minute'] = df['사고일시'].dt.minute
    df['weekday'] = df['사고일시'].dt.weekday

In [ ]:
holi_weekday = ['2019-01-01', '2019-02-04', '2019-02-05', '2019-02-06', '2019-03-01', '2019-05-05', '2019-05-12', '2019-06-06', '2019-08-15', '2019-09-12', '2019-09-13', '2019-09-14', '2019-10-03', '2019-10-09', '2019-12-25',
                '2020-01-01' ,'2020-01-24' ,'2020-01-25', '2020-01-26', '2020-03-01', '2020-04-30', '2020-05-05', '2020-06-06', '2020-08-15', '2020-08-17', '2020-09-30', '2020-10-01', '2020-10-02', '2020-10-03', '2020-10-09', '2020-12-25',
                '2021-01-01' ,'2021-02-11' ,'2021-02-12', '2021-02-13', '2021-03-01', '2021-05-05', '2021-05-19', '2021-06-06', '2021-08-15', '2021-09-20', '2021-09-21', '2021-09-22', '2021-10-03', '2021-10-09', '2021-12-25',
                '2022-01-01' ,'2022-01-31' ,'2022-02-01', '2022-02-02', '2022-03-01', '2022-05-05', '2022-05-08', '2022-06-06', '2022-08-15', '2022-09-09', '2022-09-10', '2022-09-11', '2022-09-12', '2022-10-03', '2022-10-09', '2020-10-10', '2022-12-25',
                '2023-01-01' ,'2023-01-21' ,'2023-01-22', '2023-01-23', '2023-01-24', '2023-03-01']

In [ ]:
train_df['사고일시'] = pd.to_datetime(train_df['사고일시'])
train_df['day_of_week'] = train_df['사고일시'].dt.dayofweek
train_df['holiday'] = np.where((train_df.day_of_week >= 5) | (train_df.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [ ]:
test_df['사고일시'] = pd.to_datetime(test_df['사고일시'])
test_df['day_of_week'] = test_df['사고일시'].dt.dayofweek
test_df['holiday'] = np.where((test_df.day_of_week >= 5) | (test_df.사고일시.dt.strftime('%Y-%m-%d').isin(holi_weekday)), 1, 0)

In [ ]:
train_df = train_df.drop(columns=['사고유형 - 세부분류','법규위반','가해운전자 차종','가해운전자 성별','가해운전자 연령',
                               '가해운전자 상해정도','사망자수','사망자수','중상자수','피해운전자 차종','피해운전자 성별',
                      '피해운전자 연령','피해운전자 상해정도','경상자수','부상자수','사고유형 - 세부분류'])

# 계절 변수 추가

In [ ]:
def group_season(df):
    df.loc[(df['month'] == 3) | (df['month'] == 4) | (df['month'] == 5), 'season'] = '봄'
    df.loc[(df['month'] == 6) | (df['month'] == 7) | (df['month'] == 8), 'season'] = '여름'
    df.loc[(df['month'] == 9) | (df['month'] == 10) | (df['month'] == 11), 'season'] = '가을'
    df.loc[(df['month'] == 12) | (df['month'] == 1) | (df['month'] == 2), 'season'] = '겨울'
    return df['season']

train_df['Cosine_Time'] = np.cos(2 * np.pi * train_df['hour'] / 24)
test_df['Cosine_Time'] = np.cos(2 * np.pi * test_df['hour'] / 24)
train_df['season'] = group_season(train_df)
test_df['season'] = group_season(test_df)

### 새벽 변수 추가

In [ ]:
def is_dawn(x):
    dawn_time = [0, 1, 2, 3, 4, 5]
    if x in dawn_time:
        return 1
    else:
        return 0

In [ ]:
train_df['dawn'] = train_df['hour'].map(lambda x: is_dawn(x))
test_df['dawn'] = test_df['hour'].map(lambda x: is_dawn(x))

# Target-Encoding

In [ ]:
# Target encoding
from category_encoders.target_encoder import TargetEncoder

categorical_features = ['ID','시군구','요일','도로형태','사고유형','구','기상상태', '노면상태','season']

for i in categorical_features:
    tr_encoder = TargetEncoder(cols=[i])
    train_df[i] = tr_encoder.fit_transform(train_df[i], train_df['ECLO'])
    test_df[i] = tr_encoder.transform(test_df[i])

In [ ]:
target = train_df['ECLO']

In [ ]:
train_df['ECLO'] = target

In [ ]:
train_df.corr(numeric_only=True)

ID        요일      기상상태       시군구      도로형태      노면상태  \
ID           1.000000  0.061771  0.014750  0.118496  0.086154  0.018617   
요일           0.061771  1.000000  0.005726 -0.011276  0.009456  0.003387   
기상상태         0.014750  0.005726  1.000000  0.006730  0.005441  0.811933   
시군구          0.118496 -0.011276  0.006730  1.000000  0.025979 -0.005277   
도로형태         0.086154  0.009456  0.005441  0.025979  1.000000  0.019300   
노면상태         0.018617  0.003387  0.811933 -0.005277  0.019300  1.000000   
사고유형         0.137211  0.010758 -0.019851  0.093599  0.238355 -0.008089   
ECLO         1.000000  0.061771  0.014750  0.118496  0.086154  0.018617   
구            0.117913 -0.011102  0.007334  0.995708  0.026132 -0.004705   
설치개수        -0.021686 -0.000137  0.002039 -0.188473 -0.019407  0.007760   
cnt         -0.017580  0.003156 -0.009443 -0.192218 -0.024551 -0.014793   
급지구분_1      -0.021554  0.006879  0.004096 -0.202324  0.002281  0.011160   
급지구분_2      -0.006033  0.002766  0.003445 -0.056741  0.043712  0.008992   
급지구분_3       0.006084 -0.007188 -0.000026  0.055975  0.038328 -0.004573   
군_남구        -0.008442  0.001218  0.007994 -0.074366 -0.004220  0.013111   
군_달서구       -0.019475 -0.005225  0.005361 -0.172673 -0.045384 -0.001716   
군_달성군        0.029530 -0.007254 -0.007162  0.283168 -0.004453 -0.033907   
군_동구         0.020044  0.013847 -0.003295  0.151107  0.019344 -0.001930   
군_북구        -0.005467 -0.009565  0.019264 -0.061944  0.048630  0.022963   
군_서구         0.004751  0.001974 -0.015615  0.048293  0.032176 -0.009402   
군_수성구        0.000084 -0.005951 -0.010993  0.007487 -0.034075  0.000521   
군_중구        -0.013992  0.018905  0.000075 -0.105349  0.002999  0.003103   
사고발생횟수      -0.040406 -0.001278 -0.003080 -0.353927 -0.019708  0.002751   
year        -0.031281 -0.015829  0.028432  0.013896 -0.008411  0.018757   
month       -0.006871 -0.000869 -0.019630  0.011130 -0.002795 -0.023078   
day         -0.012303 -0.001398 -0.012132  0.001173  0.006277 -0.017716   
hour        -0.017355 -0.028288  0.025395 -0.014139 -0.014810  0.019702   
minute            NaN       NaN       NaN       NaN       NaN       NaN   
weekday      0.039407  0.637956 -0.007867 -0.010625  0.008161 -0.013314   
day_of_week  0.039407  0.637956 -0.007867 -0.010625  0.008161 -0.013314   
holiday      0.060091  0.861151  0.004480 -0.009383  0.007643  0.000965   
Cosine_Time  0.037052  0.031918  0.099668 -0.046177  0.022999  0.081766   
season       0.013063  0.010607  0.019916  0.007512 -0.002123  0.016906   
dawn         0.046850  0.083368  0.040733 -0.033960  0.024622  0.034035   

                 사고유형      ECLO         구      설치개수  ...     month       day  \
ID           0.137211  1.000000  0.117913 -0.021686  ... -0.006871 -0.012303   
요일           0.010758  0.061771 -0.011102 -0.000137  ... -0.000869 -0.001398   
기상상태        -0.019851  0.014750  0.007334  0.002039  ... -0.019630 -0.012132   
시군구          0.093599  0.118496  0.995708 -0.188473  ...  0.011130  0.001173   
도로형태         0.238355  0.086154  0.026132 -0.019407  ... -0.002795  0.006277   
노면상태        -0.008089  0.018617 -0.004705  0.007760  ... -0.023078 -0.017716   
사고유형         1.000000  0.137211  0.092879 -0.029901  ...  0.000300 -0.000257   
ECLO         0.137211  1.000000  0.117913 -0.021686  ... -0.006871 -0.012303   
구            0.092879  0.117913  1.000000 -0.189628  ...  0.011202  0.001162   
설치개수        -0.029901 -0.021686 -0.189628  1.000000  ...  0.007981  0.002052   
cnt         -0.034773 -0.017580 -0.195208  0.696756  ... -0.003231  0.008446   
급지구분_1      -0.033995 -0.021554 -0.202473  0.731575  ...  0.003753  0.002438   
급지구분_2      -0.006863 -0.006033 -0.057678 -0.058398  ... -0.010281  0.002196   
급지구분_3       0.006619  0.006084  0.055171 -0.104911  ... -0.012703 -0.008205   
군_남구        -0.020346 -0.008442 -0.066767  0.803986  ...  0.008016 -0.002113   
군_달서구        0.000373 -0.019475 -0.173755 -0.184758  ...  0.000992 -0.005598   
군_달성군        0.022542  0.029530  

In [ ]:
#train_df = train_df.drop(columns=['ECLO'])

In [ ]:
#test_df = test_df.drop(columns=['사고일시'])

In [ ]:
train_df.columns

Index(['ID', '사고일시', '요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', 'ECLO', '구',
       '도시_x', '설치개수', '도시_y', 'cnt', '도시', '급지구분_1', '급지구분_2', '급지구분_3',
       '군_남구', '군_달서구', '군_달성군', '군_동구', '군_북구', '군_서구', '군_수성구', '군_중구',
       '사고발생횟수', 'year', 'month', 'day', 'hour', 'minute', 'weekday',
       'day_of_week', 'holiday', 'Cosine_Time', 'season', 'dawn'],
      dtype='object')

## 비비드백님의 인사이트를 참고하였습니다

In [ ]:
test_x = test_df.drop(columns=['ID','도시_x','도시_y','도시','year','month','day','hour','minute','day_of_week','사고일시']).copy()
train_x = train_df[test_x.columns].copy()
train_y = train_df['ECLO'].copy()

In [ ]:
test_x.columns

Index(['요일', '기상상태', '시군구', '도로형태', '노면상태', '사고유형', '구', '설치개수', 'cnt',
       '급지구분_1', '급지구분_2', '급지구분_3', '군_남구', '군_달서구', '군_달성군', '군_동구', '군_북구',
       '군_서구', '군_수성구', '군_중구', '사고발생횟수', 'weekday', 'holiday', 'Cosine_Time',
       'season', 'dawn'],
      dtype='object')

In [ ]:
test_x

요일      기상상태       시군구      도로형태      노면상태      사고유형         구  \
0      4.920811  4.712888  4.881657  5.006142  4.712878  3.817650  4.881657   
1      4.920811  4.712888  4.563008  4.653990  4.712878  3.817650  4.563008   
2      4.920811  4.712888  4.945578  5.006142  4.712878  4.944597  4.945578   
3      4.920811  4.712888  4.438172  4.653990  4.712878  4.944597  4.438172   
4      4.920811  4.712888  4.738938  5.006142  4.712878  4.944597  4.738938   
...         ...       ...       ...       ...       ...       ...       ...   
10958  4.920811  4.712888  4.492281  6.555670  4.712878  4.944597  4.492281   
10959  4.920811  4.712888  4.928144  4.653990  4.712878  4.944597  4.928144   
10960  4.920811  4.712888  5.355036  4.653990  4.712878  4.944597  5.355036   
10961  4.920811  4.712888  5.079498  4.863035  4.712878  4.944597  5.079498   
10962  4.920811  4.712888  4.840871  4.863035  4.712878  4.944597  4.840871   

         설치개수   cnt  급지구분_1  ...  군_북구  군_서구  군_수성구  군_중구  사고발생횟수  weekday  \
0       700.0   5.0     NaN  ...   0.0   0.0    1.0   0.0     338        5   
1         NaN  10.0     0.0  ...   0.0   0.0    1.0   0.0     492        5   
2         NaN   1.0     NaN  ...   0.0   0.0    1.0   0.0     147        5   
3         NaN   7.0     0.0  ...   0.0   0.0    1.0   0.0     372        5   
4       932.0   NaN     0.0  ...   0.0   0.0    0.0   0.0     678        5   
...       ...   ...     ...  ...   ...   ...    ...   ...     ...      ...   
10958  5377.0  26.0    20.0  ...   0.0   0.0    0.0   0.0    1749        5   
10959     NaN   5.0     NaN  ...   0.0   0.0    1.0   0.0     167        5   
10960     NaN   NaN     NaN  ...   0.0   0.0    1.0   0.0     107        5   
10961     NaN   7.0     NaN  ...   0.0   0.0    1.0   0.0     239        5   
10962  1164.0   NaN     NaN  ...   0.0   0.0    0.0   0.0     597        5   

       holiday   Cosine_Time    season  dawn  
0            1  9.659258e-01  4.659111     1  
1            1  9.659258e-01  4.659111     1  
2            1  5.000000e-01  4.659111     1  
3            1  5.000000e-01  4.659111     1  
4            1  6.123234e-17  4.659111     0  
...        ...           ...       ...   ...  
10958        1 -1.836970e-16  4.659111     0  
10959        1 -1.836970e-16  4.659111     0  
10960        1  5.000000e-01  4.659111     0  
10961        1  5.000000e-01  4.659111     0  
10962        1  7.071068e-01  4.659111     0  

[10963 rows x 26 columns]

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

### AUTOML

In [ ]:
import matplotlib
matplotlib.use('Agg')

In [ ]:
from supervised.automl import AutoML
automl = AutoML(mode="Compete",
                algorithms = ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
                n_jobs = -1,total_time_limit=43200, eval_metric="rmse", ml_task = "regression",)
#3.209155

In [ ]:
automl.fit(train_x, train_y)

AutoML directory: AutoML_2
The task is regression with evaluation metric rmse
AutoML will use algorithms: ['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost']
AutoML will stack models
AutoML will ensemble available models
AutoML steps: ['adjust_validation', 'simple_algorithms', 'default_algorithms', 'not_so_random', 'golden_features', 'kmeans_features', 'insert_random_feature', 'features_selection', 'hill_climbing_1', 'hill_climbing_2', 'boost_on_errors', 'ensemble', 'stack', 'ensemble_stacked']
* Step adjust_validation will try to check up to 1 model
1_DecisionTree rmse 3.224312 trained in 0.22 seconds
Adjust validation. Remove: 1_DecisionTree
Validation strategy: 10-fold CV Shuffle
Skip simple_algorithms because no parameters were generated.
* Step default_algorithms will try to check up to 4 models
1_Default_LightGBM rmse 3.234701 trained in 4.17 seconds
2_Default_Xgboost rmse 3.221246 trained in 3.53 seconds
3_Default_CatBoost rmse 3.230306 trained in 4.94 seconds
4_Default_RandomF

AutoML(algorithms=['Random Forest', 'LightGBM', 'Xgboost', 'CatBoost'],
       eval_metric='rmse', ml_task='regression', mode='Compete',
       total_time_limit=43200)

In [ ]:
pred = automl.predict(test_x)

In [ ]:
pred.max()

9.084757

In [ ]:
pred.min()

2.5187411

In [ ]:
pred

array([3.8305383, 3.4743314, 4.6353784, ..., 4.662318 , 4.6360226,
       4.671277 ], dtype=float32)

In [ ]:
submission = pd.read_csv('data/sample_submission.csv')
submission['ECLO'] = pred
submission

ID      ECLO
0      ACCIDENT_39609  3.830538
1      ACCIDENT_39610  3.474331
2      ACCIDENT_39611  4.635378
3      ACCIDENT_39612  4.293091
4      ACCIDENT_39613  4.435931
...               ...       ...
10958  ACCIDENT_50567  5.763571
10959  ACCIDENT_50568  4.402540
10960  ACCIDENT_50569  4.662318
10961  ACCIDENT_50570  4.636023
10962  ACCIDENT_50571  4.671277

[10963 rows x 2 columns]

In [ ]:
submission.to_csv('results/automl_1210.csv', index=False)